# Dependencies

In [1]:
import pandas as pd 
import os 
import re 

from datetime import datetime
from operator import itemgetter

from typing import Iterable, Union

# Overview 
In this notebook, we will read the csv files with data from all available CPCB station between Jan 2019 and Jan 2020. We clean the resulting dataframes slightly, joint each station with its correponding GPS coordinates and combine all dataframes into a single dataframe.

In [2]:
def read_cpcb(path: str, index_col: str = 'From Date') -> pd.DataFrame:
    """
    Given a path to file with data from a selected CPCB station, returns it as a conveniently parsed 
    DataFrame.
    """
    return pd.read_csv(
            path, 
            na_values=['None', ''], 
            dtype={'PM2.5': float}, 
            parse_dates = [index_col], 
            usecols = [index_col, 'PM2.5'], 
            date_parser=lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M')
        ).rename(columns={'PM2.5': 'pm2_5', index_col: 'timestamp'}).set_index('timestamp')


def read_all_cpcb(path: str) -> pd.DataFrame:
    """
    Recursively reads all files in a directory via the read_cpcb function and combines them into a single 
    DataFrame. If the given directory contains a file incompatible with the read_cpcb function, an error
    will be thrown.
    """
    return pd.concat(
        read_cpcb(path=os.path.join(root, file)).assign(site=file.split('.')[0]) 
                  for root, _, files in os.walk(path) for file in files
    )


def match_prefix(string: str, prefixes: Iterable[str]) -> Union[str, None]:
    """
    Returns prefix matching the given string from an iterable of prefixes. If not matching prefix is found, 
    returns None. 
    """
    for prefix in prefixes:
        if string.startswith(prefix):
            return prefix 


def clean_2019_jan_june(from_path: str, site_names: Iterable[str], to_path: str = None) -> None:
    """
    Transform the 2019 Jan-June data into the format consistent with the rest of the CPCB files and saves
    it at the given to_path. If no to_path is provided, to_path is set to from_path concatenated with 
    "_cleaned".
    """
    if to_path is None:
        to_path = f'{from_path}_cleaned'
    if not os.path.exists(to_path):
        os.makedirs(to_path)
    
    for file_name in os.listdir(from_path):
        df = pd.read_csv(
            os.path.join(from_path, file_name), 
            usecols=['From.Date', 'To.Date.x', 'PM2.5']
        ).rename(columns={'From.Date': 'From Date', 'To.Date.x': 'To Date'})
        
        # Save with station name matching the site locations file
        if (file_name_cleaned := match_prefix(file_name, site_names)) is not None: 
                file_name = file_name_cleaned
        df.to_csv(os.path.join(to_path, file_name))

### Cleaning 2019 Jan-June data.

In [3]:
# Replace with your path to the site locations xlsx file
site_locations_path = '../CPCB/site_locations_040619.xlsx' 
site_locations_df = pd.read_excel(site_locations_path, 
                    names=['site name', 'gpsLatitude', 'gpsLongitude', 'site'], 
                    dtype={'gpsLatitude': float, 'gpsLongitude': float}, 
                    usecols=['gpsLatitude', 'gpsLongitude', 'site'], 
                    index_col='site',
                   )
site_to_gps = site_locations_df.T.to_dict(orient='list')

# Replace with your path to the directory containing all CPCB data between Jan and June 2019
path_2019_jan_june = '../CPCB/CPCB Data/CPCB Data 2019/Jan-June'
clean_2019_jan_june(from_path=path_2019_jan_june, site_names=site_to_gps.keys())

### Merging cleaned data

In [5]:
# Replace with your path to the outer-most directory of all CPCB data
# Remember to remove the uncleaned 2019 jan-june data from this directory
path_all_data = '../CPCB/CPCB Data/'
df = read_all_cpcb(path=path_all_data).sort_index()
df = df[df['site'].isin(site_to_gps.keys())]

# Replace with your desired path where the merged data will be saved
path_merged_data = '../TF/data/CPCB_all'
df.to_csv(path_merged_data)
df

,pm2_5,site
timestamp,,
2019-01-01 00:00:00,474.50,Mundka_
2019-01-01 00:00:00,351.79,Burari_
2019-01-01 00:00:00,309.47,Noi_Sector62_
2019-01-01 00:00:00,254.00,Farida_
2019-01-01 00:00:00,361.50,Alipur_
...,...,...
2020-01-31 17:45:00,33.44,Lodhi_
2020-01-31 17:45:00,50.00,Jawaha_
2020-01-31 17:45:00,94.00,Jahang_


Adding coordinates to stations

In [6]:
df = pd.read_csv(path_merged_data, index_col='timestamp', parse_dates=['timestamp'])
df[['gpsLatitude', 'gpsLongitude']] = df['site'].map(site_to_gps).\
    transform({'gpsLatitude': itemgetter(0), 'gpsLongitude': itemgetter(1)})
df.drop(columns='site', inplace=True)

# After adding the coordinates, save in the same place 
df.to_csv(path_merged_data)
df

,pm2_5,gpsLatitude,gpsLongitude
timestamp,,,
2019-01-01 00:00:00,474.50,28.684678,77.076574
2019-01-01 00:00:00,351.79,28.725650,77.201157
2019-01-01 00:00:00,309.47,28.624548,77.357710
2019-01-01 00:00:00,254.00,28.408842,77.309908
2019-01-01 00:00:00,361.50,28.814792,77.098075
...,...,...,...
2020-01-31 17:45:00,33.44,28.591825,77.227307
2020-01-31 17:45:00,50.00,28.580280,77.233829
2020-01-31 17:45:00,94.00,28.732820,77.170633


### Combine CPCB data with DAPHNE data

In [7]:
def is_dap(file_name: str) -> bool: 
    """
    Verify that file_name represents a DAPHNE file i.e. check that the file_name starts with either "DAP" 
    or "DMC".
    """
    match = re.match(r'(?P<code>[A-Z]{3})', file_name)
    if match is None: 
        return False 
    
    return match.group('code') in ['DAP', 'DMC']


def read_dap(path: str) -> pd.DataFrame:
    """
    Read a daphne csv file from the given path. If the file name in the path does not start with 
    either "DAP" or "DMC", an error is returned. 
    """
    if not is_dap(os.path.basename(path)):
        return None 
    return pd.read_csv(
        path, 
        index_col='timestamp',
        parse_dates=['timestamp'],
        usecols = ['pm2_5', 'gpsLatitude', 'gpsLongitude', 'timestamp']
    )

In [9]:
# Replace with your path to all DAPHNE data
path_to_dap = '../TF/data/DAP_all_processed'
dap = read_dap(path=path_to_dap)
cpcb_dap_merged = pd.concat((df, dap)).sort_index()

# Replace with your desired path where the merged data will be saved
path_cpcb_dap_merged = '../TF/data/DAP_CPCB_all'
cpcb_dap_merged.to_csv(path_cpcb_dap_merged)
cpcb_dap_merged

,pm2_5,gpsLatitude,gpsLongitude
timestamp,,,
2018-08-08 05:00:00,104.549465,28.569899,77.280083
2018-08-08 05:01:00,104.525328,28.569924,77.280082
2018-08-08 05:02:00,103.990309,28.569925,77.280082
2018-08-08 05:03:00,103.235306,28.569923,77.280083
2018-08-08 05:04:00,102.500352,28.569871,77.280125
...,...,...,...
2020-02-01 06:56:00,175.675800,28.568823,77.209016
2020-02-01 06:57:00,167.558224,28.568824,77.209043
2020-02-01 06:58:00,158.351632,28.568807,77.208993
